In [ ]:
#Em cima da base do Telecom

!pip install CHAID

In [2]:
from CHAID import Tree
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Leitura dos dados
pedidos = pd.read_table("PedidoOnLine.txt")
pedidos.head()

,Idade,Genero,Estado_civil,Renda_mensal,Refeicao_mais_frequente,Avaliacao_media_anterior,Influenciado_por_nota
0,23,Feminino,Solteiro,Mais de 5.000,Lanches,"< 4,5",0
1,23,Feminino,Solteiro,Mais de 5.000,Jantar,">= 4,5",1
2,20,Feminino,Solteiro,Mais de 5.000,Café da manhã,">= 4,5",0
3,20,Feminino,Solteiro,Até 5.000,Lanches,">= 4,5",1
4,23,Masculino,Solteiro,Mais de 5.000,Jantar,">= 4,5",1


In [4]:
pedidos.dtypes

Idade                        int64
Genero                      object
Estado_civil                object
Renda_mensal                object
Refeicao_mais_frequente     object
Avaliacao_media_anterior    object
Influenciado_por_nota        int64
dtype: object

In [5]:
pedidos.shape

(2744, 7)

In [6]:
pedidos.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Idade,2744.0,28.253644,7.170123,18.0,23.0,27.0,32.0,76.0
Influenciado_por_nota,2744.0,0.708090,0.454724,0.0,0.0,1.0,1.0,1.0


In [7]:
# Verificando quantidade de missings
pedidos.isnull().sum()

Idade                       0
Genero                      0
Estado_civil                0
Renda_mensal                0
Refeicao_mais_frequente     0
Avaliacao_media_anterior    0
Influenciado_por_nota       0
dtype: int64

In [10]:
# Como a variável explicativa também deve ser categórica, 
# vamos segmentar a Idade em quartil
pedidos['Idade_q'] = pd.qcut(pedidos.Idade, 4)

In [11]:
# Tabela Bidimensional: covariável x resposta com normalize = 'index' calculando o percentual, sem esta parte, fica apenas o absoluto
Idade_tab = pd.crosstab(pedidos["Idade_q"],pedidos["Influenciado_por_nota"], normalize='index')
Idade_tab


Influenciado_por_nota,0,1
Idade_q,,
"(17.999, 23.0]",0.290609,0.709391
"(23.0, 27.0]",0.307054,0.692946
"(27.0, 32.0]",0.296721,0.703279
"(32.0, 76.0]",0.271268,0.728732


In [12]:
Genero_tab = pd.crosstab(pedidos["Genero"],pedidos["Influenciado_por_nota"], normalize='index')
Genero_tab

Influenciado_por_nota,0,1
Genero,,
Feminino,0.291139,0.708861
Masculino,0.292495,0.707505


## Exercício: Construa a tabela para as demais variáveis

In [15]:
Estado_civil_tab  =  pd.crosstab(pedidos["Estado_civil"],pedidos["Influenciado_por_nota"], normalize='index')
Estado_civil_tab

Influenciado_por_nota,0,1
Estado_civil,,
Casado,0.308989,0.691011
Outros,0.272727,0.727273
Solteiro,0.286523,0.713477


In [16]:
Renda_mensal_tab =  pd.crosstab(pedidos["Renda_mensal"],pedidos["Influenciado_por_nota"], normalize='index')
Renda_mensal_tab

Influenciado_por_nota,0,1
Renda_mensal,,
Até 5.000,0.420103,0.579897
Mais de 5.000,0.241362,0.758638


In [18]:
Refeicao_mais_frequente_tab =  pd.crosstab(pedidos["Refeicao_mais_frequente"],pedidos["Influenciado_por_nota"], normalize='index')
Refeicao_mais_frequente_tab

Influenciado_por_nota,0,1
Refeicao_mais_frequente,,
Almoço,0.196262,0.803738
Café da manhã,0.297030,0.702970
Jantar,0.255319,0.744681
Lanches,0.421875,0.578125


In [19]:
Avaliacao_media_anterior_tab =  pd.crosstab(pedidos["Avaliacao_media_anterior"],pedidos["Influenciado_por_nota"], normalize='index')
Avaliacao_media_anterior_tab

Influenciado_por_nota,0,1
Avaliacao_media_anterior,,
"< 4,5",0.583984,0.416016
">= 4,5",0.224910,0.775090


## Modelo - Árvore de Decisão

In [20]:
# Transformando a variável resposta em categórica
pedidos['resposta_cat'] = pedidos.Influenciado_por_nota.astype('category')

In [21]:
var_explicativas = pedidos[[
    'Idade_q',
    'Genero', 
    'Estado_civil',
    'Renda_mensal',
    'Refeicao_mais_frequente',
    'Avaliacao_media_anterior']]

var_resposta = pedidos['resposta_cat']

In [22]:
# Constrói o modelo de árvore
modelo = Tree.from_numpy(
    var_explicativas.to_numpy(), 
    var_resposta.to_numpy(), 
    split_titles=['Idade_q',
    'Genero', 
    'Estado_civil',
    'Renda_mensal',
    'Refeicao_mais_frequente',
    'Avaliacao_media_anterior'], 
    min_child_node_size=2)

In [23]:
modelo.print_tree()

([], {0: 801.0, 1: 1943.0}, (Avaliacao_media_anterior, p=1.913399906893216e-58, score=259.78305177131335, groups=[['< 4,5'], ['>= 4,5']]), dof=1))
|-- (['< 4,5'], {0: 299.0, 1: 213.0}, (Refeicao_mais_frequente, p=3.528025785869798e-06, score=25.109544221852858, groups=[['Almoço', 'Jantar'], ['Café da manhã'], ['Lanches']]), dof=2))
|   |-- (['Almoço', 'Jantar'], {0: 107.0, 1: 117.0}, <Invalid Chaid Split> - the max depth has been reached)
|   |-- (['Café da manhã'], {0: 96.0, 1: 64.0}, <Invalid Chaid Split> - the max depth has been reached)
|   +-- (['Lanches'], {0: 96.0, 1: 32.0}, <Invalid Chaid Split> - the max depth has been reached)
+-- (['>= 4,5'], {0: 502.0, 1: 1730.0}, (Renda_mensal, p=1.6995606630146449e-34, score=150.03945108655887, groups=[['Até 5.000'], ['Mais de 5.000']]), dof=1))
    |-- (['Até 5.000'], {0: 251.0, 1: 381.0}, <Invalid Chaid Split> - the max depth has been reached)
    +-- (['Mais de 5.000'], {0: 251.0, 1: 1349.0}, <Invalid Chaid Split> - the max depth has b

In [24]:
modelo.classification_rules()

[{'node': 2,
  'rules': [{'variable': 'Refeicao_mais_frequente',
    'data': ['Almoço', 'Jantar']},
   {'variable': 'Avaliacao_media_anterior', 'data': ['< 4,5']}]},
 {'node': 3,
  'rules': [{'variable': 'Refeicao_mais_frequente', 'data': ['Café da manhã']},
   {'variable': 'Avaliacao_media_anterior', 'data': ['< 4,5']}]},
 {'node': 4,
  'rules': [{'variable': 'Refeicao_mais_frequente', 'data': ['Lanches']},
   {'variable': 'Avaliacao_media_anterior', 'data': ['< 4,5']}]},
 {'node': 6,
  'rules': [{'variable': 'Renda_mensal', 'data': ['Até 5.000']},
   {'variable': 'Avaliacao_media_anterior', 'data': ['>= 4,5']}]},
 {'node': 7,
  'rules': [{'variable': 'Renda_mensal', 'data': ['Mais de 5.000']},
   {'variable': 'Avaliacao_media_anterior', 'data': ['>= 4,5']}]}]

In [25]:
# Salvando na base os nós
pedidos['node'] = modelo.node_predictions()
pedidos

,Idade,Genero,Estado_civil,Renda_mensal,Refeicao_mais_frequente,Avaliacao_media_anterior,Influenciado_por_nota,Idade_q,resposta_cat,node
0,23,Feminino,Solteiro,Mais de 5.000,Lanches,"< 4,5",0,"(17.999, 23.0]",0,4.0
1,23,Feminino,Solteiro,Mais de 5.000,Jantar,">= 4,5",1,"(17.999, 23.0]",1,7.0
2,20,Feminino,Solteiro,Mais de 5.000,Café da manhã,">= 4,5",0,"(17.999, 23.0]",0,7.0
3,20,Feminino,Solteiro,Até 5.000,Lanches,">= 4,5",1,"(17.999, 23.0]",1,6.0
4,23,Masculino,Solteiro,Mais de 5.000,Jantar,">= 4,5",1,"(17.999, 23.0]",1,7.0
...,...,...,...,...,...,...,...,...,...,...
2739,25,Masculino,Solteiro,Até 5.000,Café da manhã,">= 4,5",1,"(23.0, 27.0]",1,6.0
2740,37,Masculino,Casado,Mais de 5.000,Lanches,">= 4,5",1,"(32.0, 76.0]",1,7.0
2741,26,Masculino,Solteiro,Mais de 5.000,Almoço,">= 4,5",1,"(23.0, 27.0]",1,7.0
2742,36,Feminino,Solteiro,Até 5.000,Café da manhã,">= 4,5",1,"(32.0, 76.0]",1,6.0


In [26]:
# Salvando a taxa de resposta por nó
probs = pedidos.groupby(['node']).agg({'Influenciado_por_nota':'mean'}).reset_index()
probs

,node,Influenciado_por_nota
0,2.0,0.522321
1,3.0,0.400000
2,4.0,0.250000
3,6.0,0.602848
4,7.0,0.843125


In [27]:
# Marcando se o nó é propenso
probs['propenso'] = np.where(probs['Influenciado_por_nota'] >= pedidos['Influenciado_por_nota'].mean(), 1, 0)

In [28]:
probs = probs.rename(columns={"Influenciado_por_nota":"prob"})
probs

,node,prob,propenso
0,2.0,0.522321,0
1,3.0,0.400000,0
2,4.0,0.250000,0
3,6.0,0.602848,0
4,7.0,0.843125,1


In [29]:
pedidos = pedidos.merge(probs, how='left', on='node')
pedidos

,Idade,Genero,Estado_civil,Renda_mensal,Refeicao_mais_frequente,Avaliacao_media_anterior,Influenciado_por_nota,Idade_q,resposta_cat,node,prob,propenso
0,23,Feminino,Solteiro,Mais de 5.000,Lanches,"< 4,5",0,"(17.999, 23.0]",0,4.0,0.250000,0
1,23,Feminino,Solteiro,Mais de 5.000,Jantar,">= 4,5",1,"(17.999, 23.0]",1,7.0,0.843125,1
2,20,Feminino,Solteiro,Mais de 5.000,Café da manhã,">= 4,5",0,"(17.999, 23.0]",0,7.0,0.843125,1
3,20,Feminino,Solteiro,Até 5.000,Lanches,">= 4,5",1,"(17.999, 23.0]",1,6.0,0.602848,0
4,23,Masculino,Solteiro,Mais de 5.000,Jantar,">= 4,5",1,"(17.999, 23.0]",1,7.0,0.843125,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2739,25,Masculino,Solteiro,Até 5.000,Café da manhã,">= 4,5",1,"(23.0, 27.0]",1,6.0,0.602848,0
2740,37,Masculino,Casado,Mais de 5.000,Lanches,">= 4,5",1,"(32.0, 76.0]",1,7.0,0.843125,1
2741,26,Masculino,Solteiro,Mais de 5.000,Almoço,">= 4,5",1,"(23.0, 27.0]",1,7.0,0.843125,1
2742,36,Feminino,Solteiro,Até 5.000,Café da manhã,">= 4,5",1,"(32.0, 76.0]",1,6.0,0.602848,0


In [30]:
tabela_desempenho = pd.crosstab(pedidos['Influenciado_por_nota'],pedidos['propenso'])

In [31]:
tabela_desempenho

propenso,0,1
Influenciado_por_nota,,
0,550,251
1,594,1349


In [32]:
acuracia = (tabela_desempenho[0][0] + tabela_desempenho[1][1])/tabela_desempenho.sum().sum()
acuracia

0.6920553935860059

In [33]:
sensibilidade = (tabela_desempenho[1][1])/(tabela_desempenho[1][1] + tabela_desempenho[0][1])
sensibilidade

0.6942871847658261

In [37]:
#Calcule a Especificidade
especificidade = (tabela_desempenho[0][0])/(tabela_desempenho[0][0] + tabela_desempenho[1][0])
especificidade

0.686641697877653